In [23]:
import pandas as pd
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import tensorflow


from sklearn.model_selection import train_test_split, GridSearchCV


import sys
sys.path.append('../')
from scripts import featurizer_rn


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
df = pd.read_csv("../../data/dataset-procesado.csv", low_memory=False)

In [25]:
df.head()

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,D_43,D_44,B_4,...,R_28,D_139,D_140,D_144,D_145,customer_ID,S_2,D_63,D_64,target
0,0.503081,0.122690,0.060852,0.818435,0.005692,0.150860,0.002214,0.150801,0.126017,0.275044,...,0.008680,0.004475,0.001958,0.009255,0.005948,dbc39818725bb5a6693839b2cc0aa6416619e257937f3b...,2017-07-31,CO,U,0
1,0.465945,0.002313,0.164618,0.002365,1.506865,0.477885,0.495657,0.047830,0.380410,0.111988,...,0.004941,1.007453,0.005539,0.007232,0.092750,f47daf0795add3305b3b61d7246eadbe470f7d23f720f7...,2017-11-28,CO,O,1
2,0.967004,0.147385,0.025202,1.006426,0.007549,0.112134,0.005367,0.071737,0.006661,0.034550,...,0.008593,0.006886,0.002667,0.009003,0.008484,cc6156327f2bb0f6d7026c8f4bd67925d07e90aa30c3f2...,2018-01-01,CL,O,0
3,0.606902,0.178559,0.200311,0.027147,0.009911,0.128841,0.003677,0.104828,0.257234,0.417678,...,0.006294,0.007027,0.006109,0.001858,0.001150,9a563afbb91edf3b91add14584e2cbead02cadecefa6a9...,2017-03-16,CO,O,1
4,0.499094,0.478672,0.040157,0.818853,0.008475,0.147698,0.003302,0.124946,0.255405,0.399290,...,0.006820,0.000544,0.004316,0.002098,0.008200,f83df258e87389552401e86124b840eb743986f025c230...,2017-09-01,CR,O,0


In [26]:
df.shape

(14739, 123)

In [27]:
X, Y = featurizer_rn.featurizer(df)

In [28]:
X.shape

(14739, 148)

In [29]:
dataset = X.values

In [30]:
Y

0        0
1        1
2        0
3        1
4        0
        ..
14734    1
14735    0
14736    0
14737    0
14738    1
Name: target, Length: 14739, dtype: int64

In [31]:
X = dataset.astype(float)
y = Y

In [32]:
kfold = StratifiedKFold(n_splits=5, shuffle=True)

In [33]:
for train, test in kfold.split(X, y):
    X_train, X_test = X[train], X[test]
    y_train, y_test = y[train], y[test]

In [34]:
X.shape[0]

14739

In [35]:
X_train.shape[0]/X.shape[0]

0.8000542777664699

In [36]:
X_test.shape[0]/X.shape[0]

0.19994572223353008

In [37]:
##Utilizo una sigmoid porque es una clasificacion binaria en la ultima capa, 
#utilizo el optimizador de SGD porque responde mejor que adam a la hora del manejo del learning rate

def create_baseline(input_dim, opt):
    model = Sequential()
    model.add(Dense(20, input_dim=input_dim, activation='relu'))
    model.add(Dense(80, activation='relu'))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['binary_accuracy'])
    return model

In [38]:
from keras.optimizers import SGD
opt = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model_baseline = create_baseline(X_train.shape[1],opt)

In [ ]:
model_baseline.fit(X_train, y_train, epochs=1000, verbose=1, batch_size=100,validation_split=0.2)

Epoch 1/1000
95/95 [==============================] - 0s 2ms/step - loss: 0.5286 - binary_accuracy: 0.7268 - val_loss: 0.4471 - val_binary_accuracy: 0.7872
Epoch 2/1000
95/95 [==============================] - 0s 920us/step - loss: 0.4373 - binary_accuracy: 0.7884 - val_loss: 0.4179 - val_binary_accuracy: 0.8025
Epoch 3/1000
95/95 [==============================] - 0s 929us/step - loss: 0.4269 - binary_accuracy: 0.7949 - val_loss: 0.4052 - val_binary_accuracy: 0.7995
Epoch 4/1000
95/95 [==============================] - 0s 908us/step - loss: 0.4149 - binary_accuracy: 0.8015 - val_loss: 0.4148 - val_binary_accuracy: 0.7991
Epoch 5/1000
95/95 [==============================] - 0s 883us/step - loss: 0.4092 - binary_accuracy: 0.7997 - val_loss: 0.4032 - val_binary_accuracy: 0.8046
Epoch 6/1000
95/95 [==============================] - 0s 920us/step - loss: 0.4007 - binary_accuracy: 0.8095 - val_loss: 0.4745 - val_binary_accuracy: 0.7847
Epoch 7/1000
95/95 [==============================] - 

95/95 [==============================] - 0s 911us/step - loss: 0.3698 - binary_accuracy: 0.8236 - val_loss: 0.3827 - val_binary_accuracy: 0.8232
Epoch 53/1000
95/95 [==============================] - 0s 942us/step - loss: 0.3688 - binary_accuracy: 0.8277 - val_loss: 0.3834 - val_binary_accuracy: 0.8211
Epoch 54/1000
95/95 [==============================] - 0s 1ms/step - loss: 0.3696 - binary_accuracy: 0.8257 - val_loss: 0.3836 - val_binary_accuracy: 0.8220
Epoch 55/1000
95/95 [==============================] - 0s 946us/step - loss: 0.3698 - binary_accuracy: 0.8267 - val_loss: 0.3918 - val_binary_accuracy: 0.8215
Epoch 56/1000
95/95 [==============================] - 0s 973us/step - loss: 0.3672 - binary_accuracy: 0.8257 - val_loss: 0.3890 - val_binary_accuracy: 0.8169
Epoch 57/1000
95/95 [==============================] - 0s 932us/step - loss: 0.3694 - binary_accuracy: 0.8271 - val_loss: 0.4297 - val_binary_accuracy: 0.7931
Epoch 58/1000
95/95 [==============================] - 0s 963u

Epoch 104/1000
95/95 [==============================] - 0s 992us/step - loss: 0.3600 - binary_accuracy: 0.8309 - val_loss: 0.4116 - val_binary_accuracy: 0.8101
Epoch 105/1000
95/95 [==============================] - 0s 971us/step - loss: 0.3623 - binary_accuracy: 0.8306 - val_loss: 0.3987 - val_binary_accuracy: 0.8156
Epoch 106/1000
95/95 [==============================] - 0s 990us/step - loss: 0.3630 - binary_accuracy: 0.8292 - val_loss: 0.4092 - val_binary_accuracy: 0.8152
Epoch 107/1000
95/95 [==============================] - 0s 932us/step - loss: 0.3625 - binary_accuracy: 0.8313 - val_loss: 0.3844 - val_binary_accuracy: 0.8275
Epoch 108/1000
95/95 [==============================] - 0s 981us/step - loss: 0.3623 - binary_accuracy: 0.8315 - val_loss: 0.3930 - val_binary_accuracy: 0.8186
Epoch 109/1000
95/95 [==============================] - 0s 986us/step - loss: 0.3637 - binary_accuracy: 0.8296 - val_loss: 0.3828 - val_binary_accuracy: 0.8309
Epoch 110/1000
95/95 [==================

95/95 [==============================] - 0s 999us/step - loss: 0.3602 - binary_accuracy: 0.8341 - val_loss: 0.3890 - val_binary_accuracy: 0.8237
Epoch 156/1000
95/95 [==============================] - 0s 975us/step - loss: 0.3603 - binary_accuracy: 0.8302 - val_loss: 0.4070 - val_binary_accuracy: 0.8169
Epoch 157/1000
95/95 [==============================] - 0s 1ms/step - loss: 0.3579 - binary_accuracy: 0.8293 - val_loss: 0.3872 - val_binary_accuracy: 0.8258
Epoch 158/1000
95/95 [==============================] - 0s 970us/step - loss: 0.3574 - binary_accuracy: 0.8325 - val_loss: 0.3850 - val_binary_accuracy: 0.8224
Epoch 159/1000
95/95 [==============================] - 0s 1ms/step - loss: 0.3580 - binary_accuracy: 0.8301 - val_loss: 0.3876 - val_binary_accuracy: 0.8245
Epoch 160/1000
95/95 [==============================] - 0s 1ms/step - loss: 0.3580 - binary_accuracy: 0.8308 - val_loss: 0.3850 - val_binary_accuracy: 0.8237
Epoch 161/1000
95/95 [==============================] - 0s 95

95/95 [==============================] - 0s 1ms/step - loss: 0.3537 - binary_accuracy: 0.8339 - val_loss: 0.3839 - val_binary_accuracy: 0.8253
Epoch 207/1000
95/95 [==============================] - 0s 976us/step - loss: 0.3515 - binary_accuracy: 0.8347 - val_loss: 0.3856 - val_binary_accuracy: 0.8224
Epoch 208/1000
95/95 [==============================] - 0s 1ms/step - loss: 0.3538 - binary_accuracy: 0.8337 - val_loss: 0.3879 - val_binary_accuracy: 0.8207
Epoch 209/1000
95/95 [==============================] - 0s 1ms/step - loss: 0.3528 - binary_accuracy: 0.8309 - val_loss: 0.3853 - val_binary_accuracy: 0.8232
Epoch 210/1000
95/95 [==============================] - 0s 1ms/step - loss: 0.3517 - binary_accuracy: 0.8329 - val_loss: 0.4702 - val_binary_accuracy: 0.7804
Epoch 211/1000
95/95 [==============================] - 0s 1ms/step - loss: 0.3539 - binary_accuracy: 0.8368 - val_loss: 0.4123 - val_binary_accuracy: 0.8101
Epoch 212/1000
95/95 [==============================] - 0s 963us/

...

95/95 [==============================] - 0s 1ms/step - loss: 0.3466 - binary_accuracy: 0.8352 - val_loss: 0.3933 - val_binary_accuracy: 0.8194
Epoch 309/1000
95/95 [==============================] - 0s 960us/step - loss: 0.3451 - binary_accuracy: 0.8372 - val_loss: 0.3881 - val_binary_accuracy: 0.8266
Epoch 310/1000
95/95 [==============================] - 0s 943us/step - loss: 0.3449 - binary_accuracy: 0.8381 - val_loss: 0.4428 - val_binary_accuracy: 0.8071
Epoch 311/1000
95/95 [==============================] - 0s 945us/step - loss: 0.3454 - binary_accuracy: 0.8344 - val_loss: 0.4031 - val_binary_accuracy: 0.8164
Epoch 312/1000
95/95 [==============================] - 0s 983us/step - loss: 0.3460 - binary_accuracy: 0.8337 - val_loss: 0.3961 - val_binary_accuracy: 0.8177
Epoch 313/1000
95/95 [==============================] - 0s 956us/step - loss: 0.3459 - binary_accuracy: 0.8370 - val_loss: 0.3969 - val_binary_accuracy: 0.8131
Epoch 314/1000
95/95 [==============================] - 0

95/95 [==============================] - 0s 1ms/step - loss: 0.3445 - binary_accuracy: 0.8361 - val_loss: 0.3959 - val_binary_accuracy: 0.8173
Epoch 360/1000
95/95 [==============================] - 0s 987us/step - loss: 0.3430 - binary_accuracy: 0.8377 - val_loss: 0.4002 - val_binary_accuracy: 0.8139
Epoch 361/1000
95/95 [==============================] - 0s 992us/step - loss: 0.3414 - binary_accuracy: 0.8392 - val_loss: 0.4193 - val_binary_accuracy: 0.8084
Epoch 362/1000
95/95 [==============================] - 0s 1ms/step - loss: 0.3439 - binary_accuracy: 0.8382 - val_loss: 0.4121 - val_binary_accuracy: 0.8160
Epoch 363/1000
95/95 [==============================] - 0s 1ms/step - loss: 0.3439 - binary_accuracy: 0.8412 - val_loss: 0.4041 - val_binary_accuracy: 0.8203
Epoch 364/1000
95/95 [==============================] - 0s 965us/step - loss: 0.3419 - binary_accuracy: 0.8394 - val_loss: 0.4001 - val_binary_accuracy: 0.8194
Epoch 365/1000
95/95 [==============================] - 0s 1m

95/95 [==============================] - 0s 1ms/step - loss: 0.3422 - binary_accuracy: 0.8372 - val_loss: 0.3945 - val_binary_accuracy: 0.8232
Epoch 411/1000
95/95 [==============================] - 0s 1ms/step - loss: 0.3413 - binary_accuracy: 0.8385 - val_loss: 0.3943 - val_binary_accuracy: 0.8220
Epoch 412/1000
95/95 [==============================] - 0s 1ms/step - loss: 0.3404 - binary_accuracy: 0.8431 - val_loss: 0.3998 - val_binary_accuracy: 0.8194
Epoch 413/1000
95/95 [==============================] - 0s 995us/step - loss: 0.3401 - binary_accuracy: 0.8402 - val_loss: 0.4013 - val_binary_accuracy: 0.8186
Epoch 414/1000
95/95 [==============================] - 0s 972us/step - loss: 0.3427 - binary_accuracy: 0.8393 - val_loss: 0.3940 - val_binary_accuracy: 0.8203
Epoch 415/1000
95/95 [==============================] - 0s 1ms/step - loss: 0.3395 - binary_accuracy: 0.8406 - val_loss: 0.4025 - val_binary_accuracy: 0.8169
Epoch 416/1000
95/95 [==============================] - 0s 1ms/

Epoch 462/1000
95/95 [==============================] - 0s 950us/step - loss: 0.3394 - binary_accuracy: 0.8408 - val_loss: 0.3999 - val_binary_accuracy: 0.8143
Epoch 463/1000
95/95 [==============================] - 0s 983us/step - loss: 0.3375 - binary_accuracy: 0.8408 - val_loss: 0.3951 - val_binary_accuracy: 0.8164
Epoch 464/1000
95/95 [==============================] - 0s 978us/step - loss: 0.3392 - binary_accuracy: 0.8391 - val_loss: 0.4069 - val_binary_accuracy: 0.8169
Epoch 465/1000
95/95 [==============================] - 0s 981us/step - loss: 0.3387 - binary_accuracy: 0.8403 - val_loss: 0.4238 - val_binary_accuracy: 0.8101
Epoch 466/1000
95/95 [==============================] - 0s 980us/step - loss: 0.3361 - binary_accuracy: 0.8420 - val_loss: 0.4039 - val_binary_accuracy: 0.8190
Epoch 467/1000
95/95 [==============================] - 0s 978us/step - loss: 0.3372 - binary_accuracy: 0.8437 - val_loss: 0.3966 - val_binary_accuracy: 0.8173
Epoch 468/1000
95/95 [==================

95/95 [==============================] - 0s 991us/step - loss: 0.3373 - binary_accuracy: 0.8413 - val_loss: 0.4031 - val_binary_accuracy: 0.8224
Epoch 514/1000
95/95 [==============================] - 0s 943us/step - loss: 0.3353 - binary_accuracy: 0.8424 - val_loss: 0.4077 - val_binary_accuracy: 0.8020
Epoch 515/1000
95/95 [==============================] - 0s 938us/step - loss: 0.3367 - binary_accuracy: 0.8402 - val_loss: 0.4093 - val_binary_accuracy: 0.8101
Epoch 516/1000
95/95 [==============================] - 0s 925us/step - loss: 0.3366 - binary_accuracy: 0.8428 - val_loss: 0.3956 - val_binary_accuracy: 0.8164
Epoch 517/1000
95/95 [==============================] - 0s 972us/step - loss: 0.3358 - binary_accuracy: 0.8423 - val_loss: 0.3976 - val_binary_accuracy: 0.8228
Epoch 518/1000
95/95 [==============================] - 0s 1ms/step - loss: 0.3352 - binary_accuracy: 0.8416 - val_loss: 0.4072 - val_binary_accuracy: 0.8194
Epoch 519/1000
95/95 [==============================] - 0

In [ ]:
scores_train = model_baseline.evaluate(X_train, y_train)
print("\n%s: %.2f%%" % (model_baseline.metrics_names[1], scores_train[1]*100))

In [ ]:
scores_test = model_baseline.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model_baseline.metrics_names[1], scores_test[1]*100))